(section-scf)=
# Many-electron wave functions

## Import the VeloxChem module

VeloxChem is invoked with a Python module import.

In [1]:
import veloxchem as vlx
# On Windows computers activate the lines below
# import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.


## Define the system

Define a water molecule and choose a basis set.

In [2]:
# Atomic coordinates in units of Angstrom
molecule = vlx.Molecule.read_str("""
O    0.000000000000        0.000000000000        0.000000000000                         
H    0.000000000000        0.740848095288        0.582094932012                         
H    0.000000000000       -0.740848095288        0.582094932012
    """)

basis = vlx.MolecularBasis.read(molecule, "cc-pVDZ")

## SCF optimization

Perform a self-consistent field (SCF) optimization to obtain the Hartree--Fock wave function and the associated ground-state energy.

In [3]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                